In [97]:
import pandas as pd
from torch.utils.data import DataLoader
import numpy as np, torch, re
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
from torch.nn import Embedding
import torch.nn as nn
import torch.nn.functional as F


In [3]:
df=pd.read_csv('/Users/srishtysuman/PycharmProjects/NaturalLanguageProcessing/quora_question_pair_data/train.csv')
df=df.iloc[:2000]
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
1995,1995,3969,3970,"I am visiting Sri Lanka soonfor 9 days, how ca...",Do Indians hate Sri Lankans?,0
1996,1996,3971,3972,What are some good examples of 4 stanza poems?,What are some good Ilocano poems?,0
1997,1997,3973,3974,Which CPU is better I3 4th Gen or 6th Gen?,Which is better intel i5 (6th gen) or i7 (5th ...,0
1998,1998,3975,3976,What are some of the best tourist places to vi...,Where are the foremost tourist places in Chhat...,1


In [23]:
def clean(text):
    if text=='' or text is pd.isnull(text) or type(text)!=str:
        return ''
    text=text.lower()
    text=re.sub(r"([.!?])", r" \1", text)
    text=re.sub(r"[^a-zA-Z]+", r" ", text)

    return text
df=df[["question1", "question2", "is_duplicate"]]
df["question1"]=df["question1"].apply(lambda x: clean(x))
df["question2"]=df["question2"].apply(lambda x: clean(x))
df

,question1,question2,is_duplicate
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0
...,...,...,...
1995,i am visiting sri lanka soonfor days how can i...,do indians hate sri lankans,0
1996,what are some good examples of stanza poems,what are some good ilocano poems,0
1997,which cpu is better i th gen or th gen,which is better intel i th gen or i th gen,0
1998,what are some of the best tourist places to vi...,where are the foremost tourist places in chhat...,1


In [24]:
df["question1"].tolist()

['what is the step by step guide to invest in share market in india ',
 'what is the story of kohinoor koh i noor diamond ',
 'how can i increase the speed of my internet connection while using a vpn ',
 'why am i mentally very lonely how can i solve it ',
 'which one dissolve in water quikly sugar salt methane and carbon di oxide ',
 'astrology i am a capricorn sun cap moon and cap rising what does that say about me ',
 'should i buy tiago ',
 'how can i be a good geologist ',
 'when do you use instead of ',
 'motorola company can i hack my charter motorolla dcx ',
 'method to find separation of slits using fresnel biprism ',
 'how do i read and find my youtube comments ',
 'what can make physics easy to learn ',
 'what was your first sexual experience like ',
 'what are the laws to change your status from a student visa to a green card in the us how do they compare to the immigration laws in canada ',
 'what would a trump presidency mean for current international master s students on

In [62]:
class VocabularyNMapSentence:
    def __init__(self):
        self.word2index={}
        self.index2word={'SOS':1, 'EOS':2}
        self.n_word=2
        self.word2count={}
    def addSentence(self, q):
        q_map=[]
        for word in q.split(' '):
            if word in self.word2index:
                q_map.append(self.word2index[word])
            else:
                self.word2index[word]=self.n_word
                self.index2word[self.n_word]=word
                self.n_word+=1
                self.word2count[word]=0
            self.word2count[word]+=1
        return q_map
    
build_vocab_and_map=VocabularyNMapSentence()
df["q1_map"]=df["question1"].apply(lambda x: build_vocab_and_map.addSentence(x))
df["q2_map"]=df["question2"].apply(lambda x: build_vocab_and_map.addSentence(x))
df = df[df['q1_map'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
df = df[df['q2_map'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
df


,question1,question2,is_duplicate,q1_map,q2_map
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,"[5, 10]","[2, 3, 4, 5, 6, 5, 7, 8, 9, 10, 11, 12, 14]"
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0,"[2, 3, 4, 14]","[2, 113, 842, 312, 4, 439, 204, 4488, 4, 17, 1..."
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,"[19, 4, 16, 14]","[22, 23, 27, 25, 66, 1790, 6, 553, 167, 14]"
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0,"[19, 22, 23, 19, 14]","[81, 4, 69, 1390, 1390, 3, 6, 14]"
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0,"[10, 14]","[40, 3125, 113, 10, 46, 43, 14]"
...,...,...,...,...,...
1995,i am visiting sri lanka soonfor days how can i...,do indians hate sri lankans,0,"[19, 34, 1140, 714, 22, 23, 19, 3443, 286, 460...","[70, 3765, 893, 4609, 14]"
1996,what are some good examples of stanza poems,what are some good ilocano poems,0,"[2, 99, 164, 67, 296, 16, 14]","[2, 99, 164, 67, 4615, 14]"
1997,which cpu is better i th gen or th gen,which is better intel i th gen or i th gen,0,"[40, 1445, 3, 187, 19, 495, 2629, 230, 495, 26...","[40, 3, 187, 1921, 19, 495, 2629, 230, 19, 495..."
1998,what are some of the best tourist places to vi...,where are the foremost tourist places in chhat...,1,"[2, 99, 164, 16, 4, 142, 1571, 1680, 8, 1814, ...","[371, 99, 4, 1571, 1680, 10, 4616, 14]"


In [64]:
build_vocab_and_map.n_word

6073

In [65]:
ques1=df["q1_map"].tolist()
ques2=df["q2_map"].tolist()
labels=df["is_duplicate"].tolist()


In [66]:
len(ques1)

2000

In [96]:
class Dataset:
    def __init__(self, ques1, ques2, label):
        self.ques1=ques1
        self.ques2=ques2
        self.labels=label
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, index):
        return {
            "ques1": self.ques1[index],
            "ques2": self.ques2[index],
            "label": self.labels[index]
        }
    
def collate_fn(batch):
    ques1_tensor=[]
    ques2_tensor=[]
    label_tensor=[]

    for i in range(len(batch)):
        ques1_tensor.append(batch[i]["ques1"])
        ques2_tensor.append(batch[i]["ques2"])
        label_tensor.append(batch[i]["label"])
    q1_length=[len(q) for q in ques1_tensor]
    q2_length=[len(q) for q in ques2_tensor]



    return {
        "q1":ques1_tensor,
        "q2":ques2_tensor,
        "q1_length":q1_length,
        "q2_length":q2_length,
        "label":label_tensor
    }

train_data=Dataset(ques1, ques2, labels)
for data in train_data:
    print(data)
    break
train_dataloader=DataLoader(train_data, batch_size=512, collate_fn=collate_fn)
for sample in train_dataloader:
    pass

{'ques1': [5, 10], 'ques2': [2, 3, 4, 5, 6, 5, 7, 8, 9, 10, 11, 12, 14], 'label': 0}


In [108]:
class EncoderLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.embedding=Embedding(vocab_size, embedding_dim)
        self.lstm=nn.LSTM(embedding_dim, hidden_size)
        self.linear1=nn.Linear(hidden_size, hidden_size)
        self.dropout=nn.Dropout(0.1)
        self.linear2=nn.Linear(hidden_size, hidden_size)
        self.relu=nn.ReLU()
    
    def encode_sentence(self, questions, lengths):
        sorted_indices=np.flipud(np.argsort(lengths))
        lengths=np.flipud(np.sort(lengths))
        lengths=lengths.copy()

        print(questions)

        questions=[torch.LongTensor(questions[i]).to('cpu') for i in sorted_indices]        
        questions=pad_sequence(questions, batch_first=True)

        print(questions)

        embedded=self.embedding(questions).to('cpu')
        embedded=self.dropout(embedded)

        packed_sequence=pack_padded_sequence(embedded, lengths, batch_first=True)
        out, (hn, cn)=self.lstm(packed_sequence)
        unpacked, _=pad_packed_sequence(out, batch_first=True, total_length=int(lengths[0]))

        out=self.linear2(self.relu(self.linear1(unpacked)))

        unsorted_output=torch.zeros(out.size())
        for i, encoded in enumerate(out):
            unsorted_output[sorted_indices[i]]=encoded
        return unsorted_output
    
    def cosine_similarity(self, question1, question2):
        return F.cosine_similarity(question1, question2)

    def forward(self, question1, question2, q1_length, q2_length):
        # print(question1)
        encoded_q1=self.encode_sentence(question1, q1_length)
        encoded_q2=self.encode_sentence(question2, q2_length)

        similarity_score=torch.zeros(encoded_q1.size()[0]).to('cpu')

        for index in range(encoded_q1.size()[0]):
            # Sequence lenghts are being used to index and retrieve the activations before the zero padding since they were not part of original question
            q1 = encoded_q1[index, q1_length[index] - 1, :]       
            q2 = encoded_q2[index, q2_length[index] - 1, :]   
            # print(q1_length[index] - 1) 

            # print(self.manhattan_distance(q1, q2) )
            print(q1.unsqueeze(0), q2.unsqueeze(0))
            similarity_score[index] = self.cosine_similarity(q1, q2)
        return similarity_score



hidden_size=128
embedding_dim=256  
for batch in train_dataloader:
    model=EncoderLSTM(build_vocab_and_map.n_word, embedding_dim, hidden_size)
    # print(batch['q1'])
    similarity_score=model(batch['q1'], batch['q2'], batch['q1_length'], batch['q2_length'])
    print(similarity_score)
    



[[5, 10], [2, 3, 4, 14], [19, 4, 16, 14], [19, 22, 23, 19, 14], [10, 14], [19, 34, 31, 48, 55, 2, 14], [19, 14], [22, 23, 19, 31, 14], [16, 14], [23, 19, 26, 14], [8, 16, 30, 14], [22, 70, 19, 48, 81, 26, 14], [2, 23, 8, 14], [2, 14], [2, 4, 8, 94, 31, 8, 31, 10, 4, 22, 70, 8, 4, 100, 10, 14], [2, 31, 105, 14], [2, 58, 116, 14], [33, 70, 8, 66, 4, 109, 14], [33, 99, 59, 99, 123, 14], [40, 3, 4, 10, 14], [33, 70, 14], [2, 121, 8, 66, 14], [2, 99, 4, 138, 63, 123, 135, 14], [22, 3, 10, 14], [2, 58, 39, 116, 59, 19, 148, 4, 4, 99, 4, 14], [2, 99, 123, 39, 4, 160, 14], [2, 3, 14], [58, 155, 123, 14], [2, 3, 142, 8, 89, 14], [22, 63, 19, 117, 14], [2, 121, 41, 71, 113, 98, 8, 70, 14], [2, 99, 164, 117, 151, 130, 31, 59, 4, 14], [2, 16, 113, 66, 4, 8, 71, 14], [58, 4, 164, 202, 203, 14], [2, 3, 4, 142, 10, 14], [33, 70, 164, 8, 211, 14], [19, 31, 22, 23, 19, 26, 2, 63, 19, 70, 8, 124, 10, 4, 14], [69, 31, 33, 71, 62, 2, 71, 128, 8, 66, 130, 62, 2, 63, 41, 8, 238, 14], [22, 70, 182, 117, 14],

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)